In [1]:
import sys
import os

# Go up one level to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

In [2]:
import torch

In [3]:
from models.ForecastPFN import ForecastPFN

print('loading model')
model = ForecastPFN()
model.load_state_dict(torch.load('../synthetic-data/models/model.pt'))

loading model


<All keys matched successfully>

In [4]:
from types import SimpleNamespace
args = SimpleNamespace(task_name='tradition_long_term_forecast', is_training=0, model_id='ili_36_24', model='naive_mean', data='ili', root_path='../../Time-Series-Library/dataset/illness/', data_path='national_illness.csv', features='S', target='OT', freq='h', checkpoints='./checkpoints/', seq_len=36, label_len=18, pred_len=14, seasonal_patterns='Monthly', inverse=False, embed='timeF', e_layers=2, d_layers=1, factor=3, enc_in=7, dec_in=7, c_out=7, d_model=512, n_heads=8, d_ff=2048, moving_avg=25, dropout=0.1, activation='gelu', expand=2, d_conv=4, distil=True, top_k=5, num_kernels=6, p_hidden_dims=[128, 128], p_hidden_layers=2, use_dtw=False, augmentation_ratio=0, num_workers=10, itr=1, train_epochs=10, batch_size=32, patience=3, learning_rate=0.0001, des='Exp', loss='MSE', lradj='type1', use_amp=False, use_gpu=True, gpu=0, gpu_type='cuda', use_multi_gpu=False, devices='0,1,2,3')

In [5]:
from data_provider.data_factory import data_provider
test_data, test_loader = data_provider(args, 'test')

test 180


In [6]:
import numpy as np
import pandas as pd
import datetime

In [7]:
def _ForecastPFN_time_features(ts: np.ndarray):
        if type(ts[0]) == datetime.datetime:
            year = [x.year for x in ts]
            month = [x.month for x in ts]
            day = [x.day for x in ts]
            day_of_week = [x.weekday()+1 for x in ts]
            day_of_year = [x.timetuple().tm_yday for x in ts]
            return np.stack([year, month, day, day_of_week, day_of_year], axis=-1)
        ts = pd.to_datetime(ts)
        return np.stack([ts.year, ts.month, ts.day, ts.day_of_week + 1, ts.day_of_year], axis=-1)

In [8]:
from sklearn.preprocessing import StandardScaler
import time

In [9]:
# if torch.cuda.is_available():
#     print("CUDA is available! Using GPU.")
#     device = torch.device("cuda")
# else:
#     print("CUDA is not available. Using CPU.")
device = torch.device("cpu")


print(f"Device: {device}")

Device: cpu


In [10]:
def _process_tuple(x,x_mark,y_mark, 
                    model, horizon):
    """
    x: tensor of shape (n, 1)
    x_mark: tensor of shape (n, d)
    y_mark: tensor of shape (horizon, d)

    where
    n       is the input  sequence length
    horizon is the output sequence length
    d is the dimensionality of the time_stamp (5 for ForecastPFN)
    """
    if torch.all(x == x[0]):
        x[-1] += 1

    history = x.cpu().numpy().reshape(-1, 1)
    scaler = StandardScaler()
    scaler.fit(history)
    history = scaler.transform(history)

    history_mean = np.nanmean(history[-6:])
    history_std = np.nanstd(history[-6:])
    # local_scale = history_mean + history_std + 1e-4
    local_scale = max(history_mean + history_std, 1e-4)
    history = np.clip(history / local_scale, a_min=0, a_max=1)
    history = np.nan_to_num(history, nan=0.0, posinf=1.0, neginf=-1.0)

    n, d = x_mark.shape
    if n != 100:
        if n > 100:
            target = x_mark[-100:, :]
            history = torch.tensor(history[-100:], dtype=torch.float32)
        else:
            pad_len = 100 - n
            target = torch.cat([torch.zeros(pad_len, d, dtype=x_mark.dtype, device=device), x_mark], dim=0)
            history = torch.cat([
                torch.zeros(pad_len, 1, dtype=torch.float32, device=device),
                torch.tensor(history, dtype=torch.float32, device=device)
            ], dim=0)

        history = history.unsqueeze(0).repeat(horizon, 1, 1).squeeze(-1)
        ts = target.unsqueeze(0).repeat(horizon, 1, 1)
    else:
        ts = x_mark.unsqueeze(0).repeat(horizon, 1, 1)
        history = torch.tensor(history, dtype=torch.float32)

    task = torch.ones(horizon, dtype=torch.long, device=device)
    target_ts = y_mark[-horizon:, :].unsqueeze(1)
    ts = ts.long()
    target_ts = target_ts.long()


    model_input = {
        'ts': ts,
        'history': history,
        'target_ts': target_ts,
        'task': task
    }

    t1 = time.time()
    pred_vals = model(model_input)
    # print('pred_vals:', pred_vals)
    time_diff = time.time() - t1

    scaled_vals = pred_vals['result'].detach().cpu().numpy().T.reshape(-1)
    scales = pred_vals['scale'].detach().cpu().numpy().reshape(-1)
    scaled_vals = scaler.inverse_transform([scaled_vals * scales])

    return scaled_vals, time_diff

In [11]:
def _ForecastPFN_process_batch(model, batch_x, batch_y, batch_x_mark, batch_y_mark):
    preds = []
    trues = []
    for idx, (x, y, x_mark, y_mark) in enumerate(zip(batch_x, batch_y, batch_x_mark, batch_y_mark)):

        pred, time_diff = _process_tuple(
            x, x_mark, y_mark, model, args.pred_len)

        y = y.unsqueeze(-1)[-args.pred_len:, :].to(device)
        true = y.detach().cpu().numpy()
        
        preds += [pred]
        trues += [true]
    return preds, trues, time_diff

In [12]:
from utils.metrics import metric

In [13]:
test_data.data_stamp = _ForecastPFN_time_features(
    list(test_data.data_stamp_original['date']))
model.eval()
preds = []
trues = []

# torch.autograd.set_detect_anomaly(True)

# def nan_hook(module, input, output):
#     if isinstance(output, torch.Tensor) and torch.isnan(output).any():
#         print(f"NaN in {module.__class__.__name__}")

# for name, module in pretrained.named_modules():
#     module.register_forward_hook(nan_hook)

# folder_path = './test_results/' + setting + '/'
# if not os.path.exists(folder_path):
#     os.makedirs(folder_path)
# print(folder_path)
target_dim = 0
with torch.no_grad():
    for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
        # Move to device
        batch_x = batch_x.float().to(device)              # [B, seq_len, D]
        batch_y = batch_y.float().to(device)              # [B, label_len + pred_len, D]
        batch_x_mark = batch_x_mark.float().to(device)
        batch_y_mark = batch_y_mark.float().to(device)

        # Select target dimension only: shape [B, T]
        x = batch_x[:, :, target_dim]                          # [B, seq_len]
        y = batch_y[:, :, target_dim]                          # [B, label_len + pred_len]

        # ForecastPFN expects input shape [B, T], not multivariate
        pred, true, _ = _ForecastPFN_process_batch(
            model, x, y, batch_x_mark, batch_y_mark
        )

        preds.append(pred)
        trues.append(true)
        input = batch_x.detach().cpu().numpy()
        if test_data.scale and args.inverse:
            shape = input.shape
            input = test_data.inverse_transform(input.reshape(shape[0] * shape[1], -1)).reshape(shape)
        true = np.array(true)
        pred = np.array(pred)
        # print('input shape:', input.shape)
        # gt = np.concatenate((input[0, :, -1], true[0, :, -1]), axis=0)
        # pd = np.concatenate((input[0, :, -1], pred[0, :, -1]), axis=0)
        # print()
        # visual(gt, pd, os.path.join(folder_path, str(i) + '.pdf'))

preds = np.concatenate(preds, axis=0)
trues = np.concatenate(trues, axis=0)
print('test shape:', preds.shape, trues.shape)
preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])
trues = trues.reshape(-1, trues.shape[-2], trues.shape[-1])
print('test shape:', preds.shape, trues.shape)
mae, mse, rmse, mape, mspe = metric(preds, trues)
print('mse:{}, mae:{}, rmse:{}, mape:{}, mspe:{}'.format(mse, mae, rmse, mape, mspe))

test shape: (160, 1, 14) (160, 14, 1)
test shape: (160, 1, 14) (160, 14, 1)
mse:1.115865305300811, mae:0.8542831944643586, rmse:1.0563452585688122, mape:0.4291490448508656, mspe:0.4990638703813162
